RESTART kernel prior to running after any changes to config files...

### import configuration and functions modules

In [1]:
%%time
import functions as f
import list_builder as lb
import pandas as pd

import numpy as np
np.set_printoptions(linewidth=120, edgeitems=10)

CPU times: user 444 ms, sys: 24 ms, total: 468 ms
Wall time: 472 ms


### build program files
dataframe pickle files: master, fur, sg(special group), active_each_month, order proposals, month percentage

dictionary pickle files: settings_dict, color_dict

pay data pickle files: pay_table_basic, pay_table_enhanced

pay data spreadsheet workbook: pay_table_data.xlsx (in reports folder)

#### note: add case study name after script name

In [2]:
%%time
%run build_program_files sample3

CPU times: user 2.54 s, sys: 28 ms, total: 2.57 s
Wall time: 2.57 s


### skeleton

In [3]:
%%time
%run make_skeleton

CPU times: user 888 ms, sys: 180 ms, total: 1.07 s
Wall time: 1.03 s


### standalone
standalone must be rerun after job level model change...

In [4]:
%%time
%run standalone prex

CPU times: user 2.49 s, sys: 396 ms, total: 2.88 s
Wall time: 2.8 s


### calculate for each list order
(including list conditions, job count changes, and possible furlough recall schedule)

The cells below run the compute_measures script with proposals p1, p2, and p3.  All scenarios include a "prex" or pre-existing job assignment condition.  The first case includes a ratio condition and the second case includes a count-ratio condition.

In [5]:
%%time
%run compute_measures p1 prex ratio

CPU times: user 1.55 s, sys: 340 ms, total: 1.89 s
Wall time: 1.79 s


In [6]:
%%time
%run compute_measures p2 prex count

CPU times: user 1.51 s, sys: 344 ms, total: 1.86 s
Wall time: 1.76 s


In [7]:
%%time
%run compute_measures p3 prex

CPU times: user 1.42 s, sys: 416 ms, total: 1.84 s
Wall time: 1.72 s


### merge an ordered list with the master list

In [14]:
%%time
%run join_inactives p3 ffill

CPU times: user 2.22 s, sys: 8 ms, total: 2.22 s
Wall time: 2.22 s


In [15]:
%%time
f.print_config_selections()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 5.57 ms


,setting
option,
add_doh_col,1
add_eg_col,1
add_ldate_col,1
add_line_col,1
add_lname_col,1
add_ret_mark,1
add_retdate_col,1
add_sg_col,1
annual_pcnt_raise,0.02


### cells below for demonstration only

### build a 'hybrid' list example:
(stored as 'dill/hybrid.pkl')

In [16]:
%%time
# this will build a "hybrid" list with equal weighting given to longevity and standalone job position
master = lb.prepare_master_list()
lb.build_list(master, ['ldate', 'jobp'], [.5, .5])

CPU times: user 124 ms, sys: 0 ns, total: 124 ms
Wall time: 124 ms


In [17]:
%%time
# generate dataset for the hybrid list
%run compute_measures hybrid prex

CPU times: user 1.49 s, sys: 236 ms, total: 1.72 s
Wall time: 1.63 s


### sample of files created

In [18]:
%%time
# print only the first 5 lines of each file
case_study = pd.read_pickle('dill/case_dill.pkl').case.value
file_names = ['master', 'skeleton', 'standalone', 'p_p1', 'pay_table_enhanced', 'pay_table_basic', 'ds_p1', 'p_hybrid', case_study + '_final']
for file in file_names:
    try:
        df = pd.read_pickle('dill/' + file + '.pkl')
        print('< ' + file.upper() + ' >', '\n', 'row count: ' + str(len(df)),
            '\n', 'column count: '+ str(len(df.columns)), '\n',
            df.columns, '\n',
            '\n', df.head().to_string(line_width=120), '\n\n', '-' * 80, '\n')
    except:
        pass

< MASTER > 
 row count: 7510 
 column count: 11 
 Index(['empkey', 'eg', 'lname', 'dob', 'doh', 'ldate', 'sg', 'fur', 'line',
       'eg_order', 'retdate'],
      dtype='object') 
 
             empkey  eg    lname        dob        doh      ldate  sg  fur  line  eg_order    retdate
empkey                                                                                              
10011102  10011102   1  tooeyoo 1949-07-13 1973-02-26 1975-01-29   0    0     1         1 2014-07-13
10010475  10010475   1  rubelot 1949-02-05 1975-05-27 1975-05-27   0    0     1         2 2014-02-05
10013096  10013096   1  yeloxid 1949-01-08 1977-01-18 1977-01-18   0    0     1         3 2014-01-08
10012178  10012178   1  xayeaue 1951-06-07 1977-11-15 1977-11-15   0    0     1         4 2016-06-07
10014447  10014447   1  finuceu 1951-10-17 1977-12-09 1977-12-09   0    0     1         5 2016-10-17 

 -------------------------------------------------------------------------------- 

< SKELETON > 
 row count